## 用全卷积自编码器增强图像聚类识别  
  
#### 摘要  
  
  传统的图像聚类分为两步：一个是特征学习，一个是图像聚类。现有研究表明：把两步同一到一起学习，会又更好的表现。  
    
    
 这篇文章用全卷积自编码器完成特征学习和图像聚类。  
   
 使用深度网络做聚类现在大致可以分为两类：  
   
 1、在DNN学得得表示上应用聚类方法  
   
 2、联合优化深度表示学习和聚类目标的统一方法  
   
<br>  
  
* 第一类：  
  
  首先训练一个深度神经网络，用来从原生空间以一种非线性的方法嵌入到低维特征空间。之后就可以采用传统的聚类方法（如k-均值或谱聚类或凝聚聚类方法）对低维特征空间进行划分。  
    
   其缺点：由于深度学习表征的过程和聚类过程互相分离，可能导致学到的低维特征对聚类过程来说并不是特别可靠。 
    
      
* 第二类：  
  
  第二类解决了降维过程和聚类过程相互分离的问题。  这一类的思想是：好的特征表示有利于聚类结果，相反的正确的聚类结果能为特征学习提供监督信号  
    
  因此这类方法的关键是：设计一个合适的表征学习模型和设计一个合适的统一学习目标这两个因素将极大地影响这些方法的性能。  
    
<br>  
  
#### 本文的框架  
  
  首先用全卷积自编码器来快速的大体上的提取特征，然后抛弃解码器的部分，在编码器的顶端加上软的k-means模型，形成一个统一的模型。  

  
该模型与逐渐增强的识别能力联合训练。得分高的分配被突出，得分低的分配被忽视。  
  
    
<br>     
  
总结如下：  
  
1、全卷积自编码器的编码器用于快速粗疏的提取特征。  
  
  $\qquad$全卷积自编码器：由卷积层和池化层组成，且通过对每个卷积层增加批标准处理化层，可以对整个全卷积自编码器进行端到端的训练（这避免了传统堆栈(卷积)自动编码器采用的冗长和耗时的分层预训练阶段）。  
    
  
2、 基于1学的特征和软k-meansm模型的基础上，提出了增强的识别聚类框架  
  
 $\qquad$我们自定义了一个训练过程，在这个过程中，原生图片的深度特征学习和聚类的分配是联合完成的。（这步克服了分离的问题）。  
     
       
<br>  
  
####    相关工具  
  
* 堆栈式自编码器（Stacked auto-encoders）   
  
  堆栈式自编码器可以用于深度特征学习和非线性降维。其扩展方案是：卷积堆栈式自编码器（用于图片特征提取） 。  
    
  其训练分为两个步骤：1、是分层的预训练。2、全局的微调    
    
  由于是分层训练所以存在一个重大的缺陷是：耗时和冗余（尤其是基础层是受限波尔兹曼机，或者网络的层数很深时）。  
    
  对于这个问题的解决方案：最近提出了抛弃分层预训练，直接以端到端的方式训练一个深度自编码器的网络。   
    
<br>  
  
* 深度去卷积网络用于图像的分割：  
  
  输入是一个图像，输出一个图像的局部遮罩。  
    
  与同类方法相比，该网络具有最先进的性能，这要归功于三个因素：  
    
  1、引入了反卷积层和反池化层来恢复分割图像的原始尺寸。  
    
  2、将批处理归一化应用于每个卷积层和每个反卷积层，以减少内部协变量变化，这不仅使端到端的训练过程成为可能，而且加快了训练的速度  
    
  3、在大型数据集上采用预训练编码器。  
    
<br>  
  
<br>  
  
#### 实现细节  
  
框架分为两部分：  
    
1、训练全卷积自动编码器（FCAE）用来快速和粗疏的提取特征。  
  
2、在训练好FCAE之后只使用其中的编码器，再组合软k-means模型形成一个增强聚类识别模型。   
  
<br>  
  
* 训练全卷积自动编码器（FCAE）的细节：  
  
     **传统的深度卷积自编码器采用的逐层训练，之后合并，最后微调的方式，这会造成时间代价高，冗余，为了解决这个问题。**  
      
     **本文提出了：$\color{red}{可以用端到端方式训练的全卷积自动编码器}$**  
       
     1、全卷积：  
       
     2、对称性：   
       
     3、标准化：  
       
     整个网络的深度随着输入图像大小的增加而增加。如果原始图像的宽度或高度非常大，这可能会使网络非常深。为了克服这一问题，我们采用了批量归一化(BN)策略来减小内部协变量移位，加快训练速度。BN操作在每个卷积层和除最后一个输出层之外的每个反卷积层之后执行。正如在中指出的，BN对于优化全卷积神经网络至关重要  
       
<br>  
  
* 增强聚类识别的细节：  
  
  再FCAE训练好之后，我们只使用其编码器的部分，在编码器的终端添加一个软k-means 模型。  
    
  训练方法：  
    
    先用简单的样本，再到逐渐复杂的样本。（简单样本是指哪些容易确定属于哪些分类，复杂样本是指这个样本不容易判断属于哪个类别）。  
      
    要解决三个方面的问题：  
      
     1、选择什么的标准来确定哪些是容易的样本，哪些要复杂的样本。  
       
     2、怎么把复杂的样本转换为简单的样本呢？  
       
     3、怎么从简单的样本中学习聚类。  
       
    <br>  
      
    解决方案：  
      
     1、用soft k-means scores来测量简单的样本  
       
     我们使用T-student分布进行软分配，来测量简单的样本。（t分布用于解决低维数据拥挤问题）：t分布的核心是：特征z和聚类中心u的软得分（相似性）  
     ![](img/DL0132.PNG)  
       
    2、识别目标的分布来增强简单样本（复杂样本转换为简单的样本）  
      
     把复杂样本转换为简单样本，通过把加强高得分的分配，降低低得分的分配。  
       
     实现方法为：构造一个建在的目标分配r：  
       
     ![](img/DL0133.PNG)  
       
     其中s是t分布得到的软得分（相似性）。  假设我们每一次都能理想的从S中学的R:那么我们能学的一个学习链:$S^0 \rightarrow R^0 =S^1 \rightarrow R^1 =S^2 \cdots \cdots \cdots$  
       
     其中这个学习链，是递归的：把上一次学到的R,迭代为下一次S,这样在几次迭代后，得分高的分配会越来越高，得分低的分配会越来越越低。  
       
     从这个链中，可以观察到两个属性。  
       
     属性1、如果对t=0时刻任意j和j'，都有$s^0_{ij}=s^0_{ij'}$,那么在所有j和所有时刻都有$s^t_{ij}=1/K$  
       
     属性2、如果存在一个$l$,使得$s^0_{il}=\underset{j\neq l}{max} s^0_{ij'}$，那么有$\underset{t \rightarrow \infty}{lim} = s^0_{ij'}  =1 \;\;\;if\;\;\; j'=l   \qquad\qquad \underset{t \rightarrow \infty}{lim} =  s^0_{ij'}  =0 \;\;\;if\;\;\; j'\neq l $   
       
     <br>  
       
     这两个属性的作用：  
       
     属性1告诉我们，最难分的样本，是被分配到每一个聚类的概率都相同，但是这样的样本几乎很难存在。  
       
     属性2告诉我们，初始化中不可识别的样本，会逐渐变的可识别。结果就是我们把样本变得成一簇簇的，利用k-means 变的极易聚类。  
       
     注意事项：这里的$\alpha$是增强元素，控制了学习进程的速度。$\alpha$越大速度越快。然而越大越快可能导致分类不准确，特征不可靠。  
       
     <br>  
       
     3、最总的聚类目标，（进行简单样本的聚类）（使用kL散度） 
       
     我们从分布S,学的分布R,所以目标是用KL散度，让两个分布的损失最小。    
         
     损失函数：（u是聚类中心，$\theta$是分布的参数）
     
     ![](img/DL0134.PNG)  
       
     从直观上来看，损失函数有以下3点主要特征：  
       
     1、对于一个最复杂样本（分配给每个聚类的概率相同），这个样本的损失函数为：$L_i=$  
       
     ![](img/DL0135.PNG)  
       
     这个最复杂样本的损失为0，也就说明：这个最复杂样本在学习的过程中，被忽略   
       
     2、对于一个简单的好分的样本（存在$l,1>>s_{il}>\underset{j\neq l}{max}\;\;{s_{ij}}$），其损失比0大的多，所以在优化的时候，比重较大。  
       
     3、对于一个明确可以分类的样本，（存在$l,1\approx s_{il}>>\underset{j\neq l}{max}\;\;{s_{ij}}$），在相似矩阵的时候已经接近于1了。它的损失函数接近0，（意味着这个特征不需要过多的改变）  
     
     <br>  
       
     通过上面贴图的3个公式，可以求出，KL散度关于$z_j,\mu_j$的梯度
       
  ![](img/DL0136.PNG)  
  
    
      
<br>  
  
#### 算法流程  
  
  符号：T是最大的学习周期数，B是每个周期中最大的迭代更新次数，$m_b$是小批量的大小，FCAE的编码器：$f:x\rightarrow z \;\;\;$其参数是$\theta_e$，FCAE的解码器：$g:z\rightarrow x \;\;\;$其参数是$\theta_d$,  
    
<br>  
  
算法（FCAE接DBC）：  
  
  ![](img/DL0137.PNG)   
    
  ![](img/DL0138.PNG)   
    
<br>  

学习链的体现：每一个大T周期，就是一个学习链中的一个时刻。在每一个时刻中：用小批量样本来最小化$s_{ij}$和$r_{ij}$（让两个分布相等），所以下一个时刻开始时，$s_{ij}$就等于上一个时刻的$r_{ij}$
  
<br>  
  
#### 实验结果及评价     
    
      
评价指标：  
  
  1、正确率（ACC）  
    
  2、标准化互信息（NMI）
    
<br>   

  
网络体系结构：   

  